# Google Indexing Scraper

This is a simple scraper that uses the [Serper API](https://serper.dev/) to scrape the results for a given query.

In [1]:
import requests
import json
import re
import pickle
import pandas as pd
import os
from urllib.parse import urlparse

In [2]:
if os.path.exists('./results') == False:
    os.mkdir('./results')

In [3]:
# Load credentials for Serper API. For more information visit https://serper.dev/
# For obvious reasons, the credentials.json file is not included in this repo.
f = open('credentials.json', 'r')
credentials = json.load(f)

print(f'The credentials pararameters are: {credentials.keys()}')

# Entry point for the API
url = "https://google.serper.dev/search"

The credentials pararameters are: dict_keys(['Content-Type', 'X-API-KEY'])


In [4]:
def get_results(query, num=100, page=1, gl="nl", hl="nl"):
    """
    Request the Serper API for results
    """
    payload = json.dumps({
      "q": query,
      "num": num,
      "page": page,
      "gl": gl,
      "hl": hl
    })
    response = requests.request("POST", url, headers=credentials, data=payload)
    return response.json()

def save_pickle(var, path: str):
    """
    Save variable with pickle
    """
    with open(path, 'wb') as handle:
        pickle.dump(var, handle, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(path: str):
    """
    Load variable with pickle
    """
    with open(path, 'rb') as handle:
        var = pickle.load(handle)
        return var
    
def normalize_string(string):
    """
    Normalize string by removing double spaces and trailing spaces. Also capitalize the first letter of each word.
    """
    return re.sub(r' {2,}', ' ', string).strip().title()


def get_base_url(url):
    parsed_url = urlparse(url)
    base_url = f"{parsed_url.scheme}://{parsed_url.netloc}"
    return base_url

In [5]:
# Test normalize_string function
print(normalize_string('  a  b  c  d  e  f  g        h  i  j  k  l  m  n  o  p  '))

# Test get_base_url function
print(get_base_url('https://www.w3schools.com/python/default.asp'))

A B C D E F G H I J K L M N O P
https://www.w3schools.com


In [6]:
df_queries = pd.read_csv("queries.csv", index_col=False)

# Just to test if the csv is loaded correctly
print(df_queries.Query.iloc[0].title())

# Normalize Queries so we can remove duplicates
df_queries.Query = df_queries.Query.apply(lambda string: normalize_string(string))

# Remove duplicates
df_queries = df_queries[~df_queries.Query.duplicated(keep='last')].reset_index(drop=True)

df_queries.head()

Bootverhuur


,Query
0,Bootverhuur
1,Bootverhuur Sneek
2,Bootverhuur Lemmer
3,Bootverhuur Enkhuizen
4,Bootverhuur Medemblik


In [ ]:
# We will request 3 pages of results for each query with 100 results per page

num_results = 100

for query in df_queries.Query:
    there_is_more = True
    for page in range(1,4):
        # DO NOT CHANGE THIS. The path will be used to tell whether the results for a query and page have already been saved or not.
        picke_file = f"results/{query}_{page}.pickle"
        if there_is_more:
            if os.path.exists(picke_file):
                print(f"Skipping {picke_file}. Already exists.")
                continue

            print(f"Getting results for {query} page {page}")

            results = get_results(query, num=num_results, page=page)

            if len(results['organic']) < num_results - 5:
                # If the number of results is less than the number of results requested, we assume there are no more results. 5 is a arbitrary number to account for the fact that sometimes the API returns less results than requested even if there are more results.
                there_is_more = False

            # Since the API is billed per request, we save the whole response in a pickle file so if, for some reason, the script stops, we can continue where we left off. Also, initially we are not using all the data returned by the API, but we might need it in the future.
            save_pickle(results, picke_file)
        else:
            save_pickle(False, picke_file)
            print(f"Skipping {picke_file}. No more results.")

In [7]:
# Create a dataframe with the results
columns=['Query', 'Position', 'Title', 'Url', 'Snippet']
data_list = []
for query in df_queries.Query:
    for page in range(1,4):
        # DO NOT CHANGE THIS
        picke_file = f"results/{query}_{page}.pickle"
        if os.path.exists(picke_file):
            results = load_pickle(picke_file)
            if results and len(results['organic']) > 0:
                for result in results['organic']:
                    data_list.append([query, result['position'] + (page-1)*100,  result['title'], result['link'], result['snippet']])
df = pd.DataFrame(columns = columns, data = data_list)

In [8]:
# Sort by query and position
df.sort_values(by=['Query', 'Position'], inplace=True, ignore_index=True)

# Remove duplicates by query and link: overlapping results from different pages
df = df[~df[['Query', 'Url']].duplicated(keep='first')].reset_index(drop=True)

# Fix the position column after removing duplicates
df['Position'] = df.groupby('Query').cumcount() + 1

# Add base url column
df['Base Url'] = df.Url.apply(lambda url: get_base_url(url))

# Save to excel
df[['Query', 'Position','Base Url', 'Title', 'Url', 'Snippet', ]].to_excel('results.xlsx', index=False)

# Calculate the number of results per query and save to excel
df_count = df.groupby('Query')[['Url']].count().sort_values(by='Url', ascending=False)
df_count.to_excel('count.xlsx')

In [9]:
df.head()

,Query,Position,Title,Url,Snippet,Base Url
0,Bootverhuur,1,Botentehuur.nl: Dé bootverhuur website van Ned...,https://www.botentehuur.nl/,"Het grootste verhuur aanbod sloepen, motorbote...",https://www.botentehuur.nl
1,Bootverhuur,2,Bootverhuur | Boten huren begint op bootverhuu...,https://www.bootverhuur.nl/,Ontdek de grootste selectie aan bootverhuurder...,https://www.bootverhuur.nl
2,Bootverhuur,3,Boot huren bij de nr 1 in bootverhuur | Motorb...,https://www.vakantievaren.nl/,Huur een boot en gun jezelf een vaarvakantie o...,https://www.vakantievaren.nl
3,Bootverhuur,4,Boot huren? Het grootste aanbod bootverhuur in...,https://www.bootverhuurinnederland.nl/,Wil je een boot huren? Hier vind je het groots...,https://www.bootverhuurinnederland.nl
4,Bootverhuur,5,"Bootverhuur Friesland, valken, zeilboten en sl...",https://www.drijfveer.nl/bootverhuur-friesland,"Bootverhuur Friesland op toplocatie! Zeilboot,...",https://www.drijfveer.nl


In [10]:
df_count.head()

,Url
Query,
Bootverhuur,253
Bootverhuur Friese Meren,150
Bootverhuur Lauwersmeer,146
Bootverhuur Noord-Holland,142
Bootverhuur Utrecht,136
